In [3]:
from preprocess_data import load_data, get_glove

reviews_df, _ = load_data()

reviews_df.head()

Loading preprocessed Parquet files...... Done!


,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Score,Summary_glove_tokens,Text_glove_tokens,Summary_glove_tokens_np,Text_glove_tokens_np
0,914403,B0009W5KHM,AV6QDP8Q0ONK4,2,2,1341014400,GOOD FUN FILM,While most straight to DVD films are not worth...,5.0,"[112, 487, 667]","[212, 122, 1681, 4, 6131, 2802, 23, 35, 1016, ...","[112, 487, 667]","[212, 122, 1681, 4, 6131, 2802, 23, 35, 1016, ..."
1,354887,6303079709,A2I8RXJN80A2D2,0,0,1168819200,Movie Review,"I have wanted this one for sometime, also. I ...",5.0,"[603, 555]","[108, 31, 652, 27, 51, 11, 0, 292099, 108, 777...","[603, 555]","[108, 31, 652, 27, 51, 11, 0, 292099, 108, 777..."
2,1407653,B004H0M2XC,A3FHV3RV8Z12E6,0,0,1386201600,When is it a good time to Consent?,Actually this was a pretty darn good indie fil...,4.0,"[82, 10, 21, 6, 112, 77, 4, 0]","[406, 27, 30, 6, 491, 15552, 112, 12431, 0, 21...","[82, 10, 21, 6, 112, 77, 4, 0]","[406, 27, 30, 6, 491, 15552, 112, 12431, 0, 21..."
3,1377458,B003ZJ9536,A12VLTA3ZHVPUY,1,1,1348704000,TRUTH,Episodes 37 to 72 of the series press on in a ...,5.0,[1748],"[5579, 2983, 4, 3652, 5, 2, 633, 1599, 17, 7, ...",[1748],"[5579, 2983, 4, 3652, 5, 2, 633, 1599, 17, 7, ..."
4,475323,630574453X,A13NM1PES9OXVN,2,3,970012800,Intelligent and bittersweet -- stays with you,"I was really impressed with this movie, but wa...",3.0,"[6031, 3, 33699, 165, 7048, 19, 18]","[108, 30, 155, 4734, 19, 27, 0, 42, 0, 6, 308,...","[6031, 3, 33699, 165, 7048, 19, 18]","[108, 30, 155, 4734, 19, 27, 0, 42, 0, 6, 308,..."


In [4]:
import numpy as np
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Average pooling function to reduce n-gram vectors
def average_pooling(embedding_matrix):
    if len(embedding_matrix) == 0:
        return np.zeros(embedding_matrix.shape[1])  # Empty vector for padding
    return np.mean(embedding_matrix, axis=0)

def prepare_ngram_features(reviews_df, glove_embedding):
    summaries = []
    texts = []
    
    for i, row in reviews_df.iterrows():
        summary_vecs = glove_embedding(torch.tensor(row['Summary_glove_tokens_np'])).cpu().numpy()
        text_vecs = glove_embedding(torch.tensor(row['Text_glove_tokens_np'])).cpu().numpy()
        
        # Average pooling for summary and text n-grams
        summary_avg = average_pooling(summary_vecs)
        text_avg = average_pooling(text_vecs)
        
        summaries.append(summary_avg)
        texts.append(text_avg)

    # Combine summary and text features
    summary_text_features = np.hstack([summaries, texts])
    
    return summary_text_features

# Get GloVe embeddings
glove_embedding = get_glove()

# Prepare features for the SVM
X_text_features = prepare_ngram_features(reviews_df, glove_embedding)
y_ratings = reviews_df['Score'].values.astype(int)


Loading GloVe... Done!


NameError: name 'torch' is not defined